In [1]:
import pyspark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
8,application_1634288852134_0011,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{
 "conf" :
    {
        "spark.serializer" : "org.apache.spark.serializer.KryoSerializer",
        "spark.sql.legacy.parquet.int96RebaseModeInRead" : "CORRECTED",
        "spark.sql.legacy.parquet.datetimeRebaseModeInWrite" : "CORRECTED",
        "spark.sql.legacy.parquet.datetimeRebaseModeInRead" : "CORRECTED",
        "spark.sql.legacy.timeParserPolicy" : "LEGACY"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9,application_1634288852134_0012,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9,application_1634288852134_0012,pyspark,idle,Link,Link,None,✔


In [3]:
from pyspark import SparkContext, SparkConf, StorageLevel
from pyspark.sql import Window, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

import time
from functools import reduce
from datetime import datetime, timedelta

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Utils

In [4]:
def to_uri(bucket, key):
    """
    List all files under a S3 bucket
    :param bucket: (string) name of the S3 bucket
    :param key: (string) S3 key
    :return: (string) URI format
    """
    return 's3://{}/{}'.format(bucket, key)


def read_parquet_s3(app, bucket, key):
    """
    Read parquet files on s3 and return a spark dataframe
    :app: (SparkSession) spark app
    :param bucket: (string) name of the S3 bucket
    :param key: (string) S3 key
    :return: (SparkDataframe)
    """
    df = app.read.parquet(to_uri(bucket, key))
    return df


def write_parquet_s3(df, bucket, key, mode='overwrite'):
    """
    Write a SparkDataframe to parquet files on a S3 bucket
    :df: (SparkDataframe)
    :param bucket: (string) name of the S3 bucket
    :param key: (string) S3 key
    """
    df.write.parquet(to_uri(bucket, key), mode=mode)


def get_current_week():
    """
    Return current week (international standard ISO 8601 - first day of week
    is Sunday, with format 'YYYYWW'
    :return current week (international standard ISO 8601) with format 'YYYYWW'
    """
    shifted_date = datetime.today() + timedelta(days=1)
    current_week_id = int(str(shifted_date.isocalendar()[0]) + str(shifted_date.isocalendar()[1]).zfill(2))
    return current_week_id


def get_timer(starting_time):
    """
    Displays the time that has elapsed between the input timer and the current time.
    :param starting_time: (timecode) timecode from Python 'time' package
    """
    
    end_time = time.time()
    minutes, seconds = divmod(int(end_time - starting_time), 60)
    print("{} minute(s) {} second(s)".format(int(minutes), seconds))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## configs

In [5]:
bucket_clean = 'fcst-clean-prod'
bucket_refined = 'fcst-refined-demand-forecast-dev'

path_clean_datalake = 'datalake/'
path_refined_global = 'global/'

first_historical_week = 201601
first_backtesting_cutoff = 201925
current_week = get_current_week()

list_purch_org = ['Z001', 'Z008', 'Z042', 'Z066', 'Z078',  'Z098', 'Z107']

shortage_history_update = False

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load all needed clean data

In [6]:
spr = spark.table('fcst_clean_prod.f_stock_picture')
lga = read_parquet_s3(spark, bucket_clean, path_clean_datalake + 'd_listing_assortment/')
sku = read_parquet_s3(spark, bucket_clean, path_clean_datalake + 'd_sku/')
but = read_parquet_s3(spark, bucket_clean, path_clean_datalake + 'd_business_unit/')
sapb = read_parquet_s3(spark, bucket_clean, path_clean_datalake + 'sites_attribut_0plant_branches_h/')
day = read_parquet_s3(spark, bucket_clean, path_clean_datalake + 'd_day/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Apply generic filters

In [7]:
sku = sku \
    .filter(~sku['unv_num_univers'].isin([0, 14, 89, 90])) \
    .filter(sku['mdl_num_model_r3'].isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Keep only open stores
but = but \
    .filter(but['but_closed'] != 1) \
    .filter(but['but_num_typ_but'] == 7)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Temporary fix with drop duplicates waiting Otman fix
sapb = sapb \
    .filter(sapb['sapsrc'] == 'PRT') \
    .filter(sapb['purch_org'].isin(list_purch_org)) \
    .filter(current_timestamp().between(sapb['date_begin'], sapb['date_end'])) \
    .select(sapb['purch_org'], sapb['sales_org'], sapb['plant_id']) \
    .drop_duplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
day = day \
    .filter(day['wee_id_week'] >= first_historical_week) \
    .filter(day['wee_id_week'] < current_week)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# stock de ventes (out stock en transit, exposition et réservés)
spr = spr \
    .filter(spr['stt_idr_stock_type'] == 67)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
lga = lga.filter(lga['sap_source'] == 'PRT')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Get stock pictures

In [13]:
stock_picture = spr \
    .join(broadcast(sku),
          on=spr['sku_idr_sku'] == sku['sku_idr_sku'],
          how='inner') \
    .join(broadcast(day),
          on=to_date(spr['spr_date_stock'], 'yyyy-MM-dd') == day['day_id_day'],
          how='inner') \
    .join(broadcast(but),
          on=spr['but_idr_business_unit'] == but['but_idr_business_unit'],
          how='inner') \
    .join(broadcast(sapb),
          on=but['but_num_business_unit'].cast('string') == regexp_replace(sapb['plant_id'], '^0*|\s', ''),
          how='inner') \
    .select(day['wee_id_week'].alias('week_id'),
            day['day_id_day'].alias('date'),
            sapb['purch_org'],
            sapb['sales_org'],
            but['but_num_business_unit'].alias('but_id'),
            sku['mdl_num_model_r3'].alias('model_id'),
            sku['sku_idr_sku'].alias('sku_idr'),
            spr['f_quantity'].cast('int').alias('stock_quantity')) \
    .persist(StorageLevel.MEMORY_ONLY)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
print("====> counting(cache) [stock_picture] took ")
start = time.time()
stock_picture_count = stock_picture.count()
get_timer(starting_time=start)
print("[stock_picture] length:", stock_picture_count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

====> counting(cache) [stock_picture] took 
5 minute(s) 20 second(s)
[stock_picture] length: 9717511852

## Get & clean store picking

In [15]:
# Get store_picking
store_picking = lga \
    .join(broadcast(sku),
          on=sku['sku_num_sku_r3'] == lga['material_id'].cast('int'),
          how='inner') \
    .join(broadcast(sapb),
          on=regexp_replace(sapb['plant_id'], '^0*|\s', '') == regexp_replace(lga['plant_id'], '^0*|\s', ''),
          how='inner') \
    .join(broadcast(but),
          on=but['but_num_business_unit'].cast('string') == regexp_replace(lga['plant_id'], '^0*|\s', ''),
          how='inner') \
    .select(but['but_num_business_unit'].alias('but_id'),
            sku['sku_idr_sku'].alias('sku_idr'),
            to_date(lga['date_valid_from'], 'yyyy-MM-dd').alias('date_valid_from'),
            to_date(lga['date_valid_to'], 'yyyy-MM-dd').alias('date_valid_to'),
            to_date(lga['date_last_change'], 'yyyy-MM-dd').alias('date_last_change')) \

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Clean duplicates & date overlaps
def overlap_period(list_periods):
    list_periods = sorted(list_periods, key=sorter, reverse = True)
    res = [list_periods[0]]
    date_from = 0
    date_to = 1
    date_update = 2
    for a in list_periods[1:]:
        overlap = False
        for b in res:
            if a[date_update] < b[date_update] and a[date_from] <= b[date_to] and b[date_from] <= a[date_to]:
                overlap = True
        if not overlap:
            res.append(a)
    return res


overlap_period_udf = udf(lambda list_periods: overlap_period(list_periods), ArrayType(ArrayType(DateType())))
sorter = lambda x: (x[2], x[0], x[1])

w_valid_to = Window().partitionBy('but_id', 'sku_idr', 'date_valid_to').orderBy(col('date_last_change').desc())

store_picking = store_picking \
    .withColumn('rn', row_number().over(w_valid_to)) \
    .filter(col('rn') == 1)
    
w_valid_from = Window().partitionBy("but_id", "sku_idr", "date_valid_from").orderBy(col("date_last_change").desc())

store_picking = store_picking\
    .withColumn("rn", row_number().over(w_valid_from)) \
    .where(col("rn") == 1)

# Note: you don't have to understand, but it works, trust me
store_picking = store_picking \
    .withColumn('dates_from_to', array(store_picking['date_valid_from'], store_picking['date_valid_to'],store_picking['date_last_change'])) \
    .groupBy("but_id", "sku_idr") \
    .agg(collect_list(col("dates_from_to")).alias("dates_from_to")) \
    .withColumn("all_periods", overlap_period_udf(col("dates_from_to"))) \
    .withColumn("period", explode(col("all_periods"))) \
    .select(col("but_id"),
            col("sku_idr"),
            col('period'),
            col('period')[0].alias("date_valid_from"),
            col('period')[1].alias("date_valid_to")) \
    .persist(StorageLevel.MEMORY_ONLY)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print("====> counting(cache) [store_picking] took ")
start = time.time()
store_picking_count = store_picking.count()
get_timer(starting_time=start)
print("[store_picking] length:", store_picking_count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

====> counting(cache) [store_picking] took 
4 minute(s) 13 second(s)
[store_picking] length: 445979432

## Calculate shortage rate per week

In [18]:
l_week_id = day.select('wee_id_week').drop_duplicates().orderBy('wee_id_week').collect()
l_week_id = [int(w[0]) for w in l_week_id]

if not shortage_history_update:
    l_week_id = l_week_id[-8:]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
start = time.time()
#l_week_id = [202110,202109,202108,202107,202106,202105,202104,202103,202102,202101]
l_week_id = [202110,202109]
for week_id in l_week_id:

    # calculate all daily_stock combinations
    all_week_date = day \
        .filter(day['wee_id_week'] == week_id) \
        .select(day['wee_id_week'].alias('week_id'), 
                day['day_id_day'].alias('date'))

    all_stock_picture_key = stock_picture \
        .filter(day['week_id'] == week_id) \
        .select(['purch_org', 'sales_org', 'but_id', 'model_id', 'sku_idr']) \
        .drop_duplicates()

    all_daily_stock_comb = all_week_date.crossJoin(all_stock_picture_key)

    # Left join with stock_picture
    daily_stock = all_daily_stock_comb \
        .join(stock_picture, 
              how='left',
              on=['week_id', 'date', 'purch_org', 'sales_org', 'but_id', 'model_id', 'sku_idr'])

    # ffill ("from delta stock check to stock by day")
    window = Window.partitionBy(['but_id', 'sku_idr']) \
        .orderBy('date') \
        .rowsBetween(-sys.maxsize, 0)

    daily_stock = daily_stock \
        .withColumn('stock_quantity', last(daily_stock['stock_quantity'], ignorenulls=True).over(window)) \
        .dropna()

    # Keep only stock combinations matching with stores picking
    daily_stock = daily_stock \
        .join(store_picking,
              on=(daily_stock['but_id'] == store_picking['but_id']) &
                 (daily_stock['sku_idr'] == store_picking['sku_idr']) &
                 (daily_stock['date'].between(store_picking['date_valid_from'], store_picking['date_valid_to'])),
              how='inner'
             ) \
        .select(daily_stock['*'])

    ## Calculate shortage rate
    # Agg day to week
    shortage_rate = daily_stock \
        .groupBy(['week_id', 'but_id', 'model_id', 'sku_idr']) \
        .agg(count(when(daily_stock['stock_quantity'] == 0, 'stock_quantity')).alias('nb_day_zero'),
             count(daily_stock['stock_quantity'] == 0).alias('nb_day_follow'))

    # Agg sku to model
    shortage_rate = shortage_rate \
        .groupBy(['week_id', 'but_id', 'model_id']) \
        .agg(sum('nb_day_zero').alias('nb_day_zero'),
             sum('nb_day_follow').alias('nb_day_follow'))

    # Agg bu to zd
    shortage_rate = shortage_rate \
        .groupBy(['week_id', 'model_id']) \
        .agg((sum('nb_day_zero') / sum('nb_day_follow')).alias('shortage_rate'))

    # Write parquet by week in batch
    write_parquet_s3(shortage_rate \
                         .filter(shortage_rate['week_id'] == week_id) \
                         .select('model_id', 'week_id', 'shortage_rate'),
                     bucket_refined,
                     path_refined_global + 'model_week_shortage_per_week/' + str(week_id))

get_timer(starting_time=start)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10 minute(s) 23 second(s)

## Read/Write model week shortage

In [ ]:
model_week_shortage = read_parquet_s3(spark, 
                                      bucket_refined, 
                                      path_refined_global + 'model_week_shortage_per_week/*/')

write_parquet_s3(model_week_shortage.orderBy('model_id', 'week_id'), 
                 bucket_refined,
                 path_refined_global + 'model_week_shortage')

In [ ]:
spark.stop()